# Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from Kaggle, and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from Kaggle
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.

   - Remember that for multi-class classification you need a softamx activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Trian your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaning the activation functions on the hidden layers, etc.
8. Load in Kaggle's `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

---

For this lab, you should complete the above sequence of steps for _at least_ two of the three "configurations":

1. Using a `tensorflow` network
2. Using a `keras` "sequential" network
3. Using a `keras` convolutional network
4. Using a `tensorflow` convolutional network (we did _not_ cover this in class!)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import numpy as np
# For reproducibility
np.random.seed(42)
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

W0819 14:29:44.094480 140735660290944 deprecation.py:323] From <ipython-input-1-0752bfcbca3c>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0819 14:29:44.095607 140735660290944 deprecation.py:323] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0819 14:29:44.097072 140735660290944 deprecation.py:323] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated an

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0819 14:29:44.426552 140735660290944 deprecation.py:323] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0819 14:29:44.429156 140735660290944 deprecation.py:323] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0819 14:29:44.513164 140735660290944 deprecation.py:323] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train.shape

(42000, 785)

In [4]:
X = train.drop(columns='label')
y = train.label

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
# When dealing with image data, you need to normalize your X by dividing each value by the max value of a pixel (255).
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train / 255
X_test = X_test / 255

X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)
# Since this is a multiclass classification problem, keras needs y to be a one-hot encoded matrix
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [7]:
y_train.shape

(31500, 10)

In [8]:
y_test.shape

(10500, 10)

## Tensorflow model

In [9]:
n_input = X_train.shape[1]  # input layer (28x28 pixels)
n_hidden1 = 512  # 1st hidden layer
n_hidden2 = 256  # 2nd hidden layer
n_hidden3 = 128  # 3rd hidden layer
n_output = 10  # output layer (0-9 digits)

learning_rate = 0.5
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

In [10]:
# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                         + (1 - y) * tf.log(1 - y_clipped), axis=1))
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

W0819 14:29:51.647398 140735660290944 deprecation.py:323] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# start the session
with tf.Session() as sess:
   # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))
 #   pred = sess.run(y_clipped, feed_dict={X:X_test})

Epoch: 1 cost = 0.718
Epoch: 2 cost = 0.263
Epoch: 3 cost = 0.202
Epoch: 4 cost = 0.160
Epoch: 5 cost = 0.134
Epoch: 6 cost = 0.114
Epoch: 7 cost = 0.096
Epoch: 8 cost = 0.084
Epoch: 9 cost = 0.073
Epoch: 10 cost = 0.063
0.9775


TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

##  Sequentual Keras Network


In [13]:
model = Sequential()
model.add(Flatten())
model.add(Dense(128, input_shape=(28, 28), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0819 14:32:03.231768 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 14:32:03.236055 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [14]:
history = model.fit(X_train,
                    y_train.values, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    verbose=1)

W0819 14:32:05.030380 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 14:32:05.053250 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 14:32:05.140084 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0819 14:32:05.340197 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.as

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 5s 148us/step - loss: 0.6463 - acc: 0.8127 - val_loss: 0.2953 - val_acc: 0.9169
Epoch 2/10
31500/31500 [==============================] - 2s 62us/step - loss: 0.2492 - acc: 0.9297 - val_loss: 0.2199 - val_acc: 0.9359
Epoch 3/10
31500/31500 [==============================] - 2s 57us/step - loss: 0.1888 - acc: 0.9470 - val_loss: 0.1791 - val_acc: 0.9495
Epoch 4/10
31500/31500 [==============================] - 2s 57us/step - loss: 0.1517 - acc: 0.9574 - val_loss: 0.1547 - val_acc: 0.9553
Epoch 5/10
31500/31500 [==============================] - 2s 58us/step - loss: 0.1291 - acc: 0.9641 - val_loss: 0.1391 - val_acc: 0.9611
Epoch 6/10
31500/31500 [==============================] - 2s 56us/step - loss: 0.1070 - acc: 0.9698 - val_loss: 0.1283 - val_acc: 0.9621
Epoch 7/10
31500/31500 [==============================] - 2s 57us/step - loss: 0.0895 - acc: 0.9756 - val_loss: 0.1182 - val_ac

In [15]:
test = pd.read_csv('test.csv')
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
test = test.astype('float32')
test /= 255
test = test.values.reshape(test.shape[0], 28, 28, 1)
preds = model.predict_classes(test)

In [17]:
preds_df = pd.DataFrame(preds, columns = ['Label'])
preds_df.index.name = 'ImageID'
preds_df.index += 1
preds_df.head()

,Label
ImageID,
1,2
2,0
3,9
4,9
5,3


In [18]:
preds_df = pd.DataFrame(preds, columns = ['Label'])
preds_df.index.name = 'ImageID'
preds_df.index += 1

In [19]:
preds_df.head()

,Label
ImageID,
1,2
2,0
3,9
4,9
5,3


In [20]:
preds_df.tail()

,Label
ImageID,
27996,9
27997,7
27998,3
27999,9
28000,2


In [ ]:
preds_df.to_csv('first_sub.csv')

## Keras CNN Network

In [21]:
cnn_model = Sequential()
cnn_model.add(Conv2D(filters = 6,            # number of filters
                     kernel_size = 3,        # height/width of filter
                     activation='relu',      # activation function 
                     input_shape=(28,28,1))) # shape of input (image)
cnn_model.add(MaxPooling2D(pool_size=(2,2))) # dimensions of region of pooling
cnn_model.add(Conv2D(16,
                     kernel_size=3,
                     activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(10, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

W0819 14:33:10.070136 140735660290944 deprecation_wrapper.py:119] From /Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [22]:
history = cnn_model.fit(X_train,
                    y_train.values, 
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    verbose=1)

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 18s 573us/step - loss: 0.7834 - acc: 0.8060 - val_loss: 0.2494 - val_acc: 0.9297
Epoch 2/10
31500/31500 [==============================] - 15s 474us/step - loss: 0.1759 - acc: 0.9471 - val_loss: 0.1489 - val_acc: 0.9574
Epoch 3/10
31500/31500 [==============================] - 15s 469us/step - loss: 0.1148 - acc: 0.9655 - val_loss: 0.0988 - val_acc: 0.9715
Epoch 4/10
31500/31500 [==============================] - 15s 470us/step - loss: 0.0887 - acc: 0.9728 - val_loss: 0.0812 - val_acc: 0.9766
Epoch 5/10
31500/31500 [==============================] - 15s 470us/step - loss: 0.0752 - acc: 0.9773 - val_loss: 0.0807 - val_acc: 0.9758
Epoch 6/10
31500/31500 [==============================] - 15s 478us/step - loss: 0.0633 - acc: 0.9809 - val_loss: 0.0646 - val_acc: 0.9805
Epoch 7/10
31500/31500 [==============================] - 15s 468us/step - loss: 0.0589 - acc: 0.9813 - val_loss: 0.

In [23]:
preds = cnn_model.predict_classes(test)

In [24]:
preds_cnn_df = pd.DataFrame(preds, columns = ['Label'])
preds_cnn_df.index.name = 'ImageID'
preds_cnn_df.index += 1
preds_cnn_df.head()

,Label
ImageID,
1,2
2,0
3,9
4,9
5,3


In [25]:
preds_df.to_csv('second_sub.csv')